In [24]:
import rebound
from astropy.time import Time
import numpy as np
from datetime import datetime, timedelta
# Función para convertir el formato comprimido de la época a fecha juliana
def convertir_epoch_comprimido(epoch_comprimido):
    letra_año = {chr(i): 1800 + (i - ord('A')) for i in range(ord('A'), ord('Z') + 1)}
    mes_map = {'1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9,
               'A': 10, 'B': 11, 'C': 12}
    dia_map = {str(i): i for i in range(1, 10)}
    dia_map.update({chr(i): 10 + (i - ord('A')) for i in range(ord('A'), ord('Z') + 1)})
    
    letra_año_epoch = epoch_comprimido[0]
    mes_epoch = epoch_comprimido[1]
    dia_epoch = epoch_comprimido[2]
    
    año = letra_año.get(letra_año_epoch, 0)
    mes = mes_map.get(mes_epoch, 0)
    dia = dia_map.get(dia_epoch, 0)
    
    if año == 0 or mes == 0 or dia == 0:
        raise ValueError(f"Formato de época no válido: {epoch_comprimido}")
    
    # Convertir a fecha juliana usando astropy
    fecha = Time(f"{año}-{mes:02d}-{dia:02d}", format='iso')
    return fecha.jd

# Función para leer el archivo MPCORB.DAT
def leer_mpcorb(archivo):
    cuerpos = []
    with open(archivo, 'r') as f:
        encabezado=True
        for linea in f:
            if '---' in linea:
                encabezado=False
                continue
            if encabezado==True:
                continue    
            if linea.startswith('#') or len(linea)==0:  # Ignorar líneas de comentario
                continue
            # Extraer elementos orbitales
            nombre = linea[0:7].strip()
            epoch_comprimido = linea[20:25].strip()  # Época en formato comprimido
            M = float(linea[26:36].strip())  # Anomalía media (grados)
            omega = float(linea[37:47].strip())  # Argumento del perihelio (grados)
            Omega = float(linea[48:58].strip())  # Longitud del nodo ascendente (grados)
            i = float(linea[60:68].strip())  # Inclinación (grados)
            e = float(linea[70:79].strip())  # Excentricidad
            n = float(linea[80:90].strip())  # Movimiento medio (grados/día)
            a = float(linea[92:103].strip())  # Semieje mayor (AU)
            
            # Convertir la época a fecha juliana
            try:
                epoch = convertir_epoch_comprimido(epoch_comprimido)
            except ValueError as e:
                print(f"Error en la línea: {linea.strip()}")
                print(e)
                continue
            
            # Guardar los datos del cuerpo
            cuerpo={
                'nombre': nombre,
                'a': a,
                'e': e,
                'i': i,
                'omega': omega,
                'Omega': Omega,
                'M': M,
                'epoch': epoch
            }
            print(cuerpo)
            cuerpos.append(cuerpo)
    return cuerpos

# Función para agregar perturbadores (Sol, Tierra, Luna y planetas)
def agregar_perturbadores(sim):
    # Agregar el Sol
    sim.add("Sun")
    
    # Agregar planetas y la Luna
    sim.add("Mercury")
    sim.add("Venus")
    sim.add("Earth")
    sim.add("Moon")  # Luna
    sim.add("Mars")
    sim.add("Jupiter")
    sim.add("Saturn")
    sim.add("Uranus")
    sim.add("Neptune")

# Función para propagar la órbita a una fecha específica
def propagar_orbita(sim, fecha):
    sim.move_to_hel()
    sim.integrate(fecha)
    orbit = sim.particles[0].orbit()
    return orbit

# Función principal
def main():
    # Cargar el archivo MPCORB.DAT
    archivo_mpcorb = 'MPCORB_TEST.DAT'
    cuerpos = leer_mpcorb(archivo_mpcorb)

    # Fechas pasadas para propagar (en días Julianos)
    fechas_pasadas = [
        2459580.5,  # Ejemplo: 1 de enero de 2021
        2459200.5,  # Ejemplo: 1 de enero de 2020
        2458800.5   # Ejemplo: 1 de enero de 2019
    ]

    # Archivo de salida
    archivo_salida = 'propagacion_orbital.txt'
    with open(archivo_salida, 'w') as f:
        f.write("Nombre,Fecha,a,e,i,omega,Omega,M\n")  # Encabezado
        sim = rebound.Simulation()
        # Agregar perturbadores (Sol, Tierra, Luna y planetas)
        agregar_perturbadores(sim)

        # Iterar sobre cada cuerpo
        for cuerpo in cuerpos:
            sim.add(
                m=0,  # Masa del cuerpo (0 para asteroides)
                a=cuerpo['a'],
                e=cuerpo['e'],
                inc=np.radians(cuerpo['i']),
                omega=np.radians(cuerpo['omega']),
                Omega=np.radians(cuerpo['Omega']),
                M=np.radians(cuerpo['M'])
            )

        sim.dt = 0.001  # Paso de tiempo (ajustar según sea necesario)

        # Propagar a cada fecha pasada
        for fecha in fechas_pasadas:
            orbit = propagar_orbita(sim, fecha)
            f.write(f"{cuerpo['nombre']},{fecha},{orbit.a},{orbit.e},{np.degrees(orbit.inc)},{np.degrees(orbit.omega)},{np.degrees(orbit.Omega)},{np.degrees(orbit.M)}\n")

    print(f"Propagación completada. Resultados guardados en {archivo_salida}.")

main()

{'nombre': '00001', 'a': 2.7660512, 'e': 0.0794013, 'i': 10.5878, 'omega': 73.27343, 'Omega': 80.25221, 'M': 188.70269, 'epoch': np.float64(2382183.5)}
{'nombre': '00002', 'a': 2.7701937, 'e': 0.2305404, 'i': 34.92402, 'omega': 310.91037, 'Omega': 172.8953, 'M': 168.7987, 'epoch': np.float64(2382183.5)}
{'nombre': '00003', 'a': 2.6706696, 'e': 0.2559472, 'i': 12.98651, 'omega': 247.85972, 'Omega': 169.82994, 'M': 172.45499, 'epoch': np.float64(2382183.5)}
{'nombre': '00004', 'a': 2.3613975, 'e': 0.0901071, 'i': 7.14394, 'omega': 151.58335, 'Omega': 103.70297, 'M': 332.45069, 'epoch': np.float64(2382183.5)}
{'nombre': '00005', 'a': 2.5767618, 'e': 0.1874407, 'i': 5.35887, 'omega': 359.31775, 'Omega': 141.45353, 'M': 86.24723, 'epoch': np.float64(2382183.5)}
{'nombre': '00006', 'a': 2.4258792, 'e': 0.2023356, 'i': 14.73542, 'omega': 239.67015, 'Omega': 138.6144, 'M': 300.41267, 'epoch': np.float64(2382183.5)}
{'nombre': '00007', 'a': 2.3863214, 'e': 0.2299781, 'i': 5.51956, 'omega': 145.

ValueError: Orbital elements for particle[0] not implemented unless primary is provided